# Temperature/Humidity Sensor on Raspberry Pi 

The following document highlights several steps taken to set up the temperature/humidity sensor on the Raspberry Pi. The document is written chronologically in order of the steps taken in setting up the project but can be read in any order as is pertinent.

## Overview of Project
In this project, a Raspberry Pi is equipped with a temperature and humidity sensor for the purpose of identifying anomalies in an Internet network in which the device is a part of. This is an example of an IOT device that will soon become commonplace in society. This project involves setting up the Raspberry Pi as well as generating traffic to/from it and finally employing different visualization techniques to identify normal and anomalous traffic involving the device. The traffic involving the device is generated by connecting the device to a network and polling for temperature/humidity information from the web server that is set up on the device as well as via SSH requests.

### Components
- Raspberry Pi 2
- DHT 11 temperature/humidity sensor


## Sensor Connections

The sensor is connected to the GPIO pins of the Raspberry Pi as follows:

GND -> GPIO Pin 6 (Ground)

VCC -> GPIO Pin 1 (3.3V)

DAT -> GPIO Pin 7 (GPIO 4)

## Raspberry Pi Configuration

Login information for the RPI  

raspberrypi login: pi  
password: raspberry  

Enter the following command at the prompt to access the GUI on the RPI:

```BASH
sudo startx
```

## Wi-Fi Setup

To connect the RPI to the internet wirelessly, ensure that a wireless adapter is connected to the device and then navigate to the Wi-Fi configuration menu as shown.  
>Menu->Preferences->Wi-Fi Configuration  


## SSH

SSH has been configured on the RPI. To use ssh with the RPI, first ensure that the remote computer is on the same network as the RPI. Next, log in to the "pi" user account on the SSH using:  

```BASH
sudo su pi
```

Then from the remote computer, enter:  
ssh pi@ipaddress

ssh login_name: pi  
ssh password: raspberry

## Sensor Software Configuration

- Details of the setup can be found at http://www.modmypi.com/blog/am2302-temphumidity-sensor  

The main steps taken were to:
- Install the Adafruit_DHT  python module from github
- Install Python DHT library and its dependencies  
These 2 steps are expounded on in the link above

To test the setup, change directories to the /home/pi/Adafruit_Python_DHT/examples folder and run:  
```BASH
sudo ./AdafruitDHT.py 11 4
```
- 11 represents the type of temperature sensor (DHT11, DHT22, AM2302)
- 4 represents the GPIO pin the sensor is connected to  
The output should be of the form: Temp=77.0* Humidity=34.0%

## Shell Scripts

A shell script was written to periodically query the sensor and store the time of query as well as the
temperature and humidity in a text file then subsequently to a database. 
- The shell script is located at /home/pi/tempsens

Shell Script Pseudocode:
- Query the sensor for current temperature and humidity readings
- Add a timestamp to the result and pipe output to a temporary file
- Use the sed tool to remove unwanted characters in the result and pipe output to target file  
Note: The name of the target file must be the same as the table in the MySQL database 
- Connect to to MySQL database and upload current line containing timestamp and sensor readings (See Database Setup and Configuration Section)
- Run script as a cron job to run periodically in the background (See Cron section)


## Cron

Cron is a job scheduler used to periodically run commands and scripts at fixed intervals. 

A cron job is used in this project to periodically run the tempsens shell script which simply polls the sensor and pushes data to the databases.

The cronjobs running on a Unix machine can be viewed by running the command:  
```BASH
sudo crontab -e
```
The above command could be run without the "sudo" but for this implementation, sudo is required. The result should be:
```BASH
*/2 * * * * /home/pi/tempsens >> /home/pi/cron.log 2>&1
```
What this command does is to run the tempsens script every two minutes and output any error messages to cron.log

## Database Setup and Configuration

A MySQL database is set up on the RPI. A quick primer on MySQL can be found at http://dev.mysql.com/doc/refman/5.7/en/database-use.html.  
The MySQL server is started by default when the Raspberry Pi is turned on. 
This can be confirmed with the command: 
```BASH
sudo service mysql status
```
To start the database service, run the command:
```BASH
sudo service mysql start
```

To logon to the database, enter the following at the terminal prompt;
```BASH
sudo mysql -u root -p
```
The information for the sensor database is shown below:  
Login_name: root  
Password: raspberry  
Database_Name: sensor  
Table_Name: readings  

To see a list of databases on the server, use the command:
```SQL
show databases;
```
To select one of the databases, use the command:
```SQL
use Database_name; 
```
To view the tables in a database, use the command:
```SQL
show tables;
```
Note: When storing data from a file, make sure to use the same name for the table as the filename.
For example, the name of the table containing the sensor data is "readings" and the name of the text file
where the data is being pulled from is called "readings.txt".

To view all the entries in a database table, use the command:
```SQL
select * from Table_name;
```
To reset database table, use the command:
```SQL
truncate Table_name;
```


## Web Server 

An apache web server is configured on the RPI.  
This command is used to check if the installed apache webserver is running:
```BASH
sudo service apache2 status
```

To serve up a webpage, an html or php file should be placed in the /var/www/ folder on the RPI. This is demonstrated below.


## WebPage Configuration

The purpose of setting up the web server above is to enable temp/humidity information from the RPI to be accessed from any device on the RPI's network via a webpage. The webpage is stored on the RPI and is essentially a page displaying the current temperature/humidity reading from the RPI. The webpage also contains a link to a historical view of the sensor's data.

The php files containing the webpage are well commented and are located at:  
/var/www/sensor.php  
/var/www/sensor2.php

The webpage can be accessed at "ipaddress/sensor.php" where ipaddress is the current ip of the raspberry pi

## SSH Connection 

Temperature and Humidity information is also accessed from the RPI using ssh connections from remote connections. 
This is done by logging in to the RPI via SSH and running the "/home/pi/temprequest" script as shown.

ssh pi@ip_address_of_RPI /home/pi/temprequest

## Getting Network Traffic

The network traffic to and from the RPI can be obtained using a packet sniffer (Wireshark, TCPdump, tshark). To get data for analysis, the packet sniffer should be started before accessing the webpage and getting readings via ssh. This traffic will most likely be in a .pcap/.pcapng format and should be converted to a different format, ideally csv, before data analysis.

## Data Analysis Using Python

The data obtained from the network traffic can be analysed using the Python packages for data analysis. An example of this is shown in the accompanying IPython notebook (EVRL VizNet).